In [ ]:
# import libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

## Similaridad coseno
¿Cómo calcularla en Python?

Podemos calcular la similaridad  coseno empleando sklearn:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
Juan = [5,4,4]
Diego = [4,5,5]
cosine_similarity([Juan, Diego])

También podemos calcular la similaridad a mano:

In [ ]:
(5*4 + 4*5 + 4*5)/(np.sqrt(5**2+4**2+4**2)*np.sqrt(4**2+5**2+5**2))

O empleando Numpy

In [ ]:
np.dot(Juan,Diego)/np.dot(np.linalg.norm(Juan), np.linalg.norm(Diego))

## Item-Based Collaborative Filtering

Retomando el ejemplo de las diapositivas:

In [ ]:
user_item = np.array([[5, np.nan, 4],[4,3,5],[4,5,5],[np.nan, 5, np.nan], [np.nan, 5, 3]])

In [ ]:
user_item

En este ejemplo vamos a calcular la similitud coseno considerando solo los valores distintos de nan. 
Vamos a usar numba:
"Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN. "

In [ ]:
import numba

@numba.jit(target='cpu', nopython=True)
def fast_cosine(u, v):
    m = u.shape[0]
    udotv = 0
    u_norm = 0
    v_norm = 0
    for i in range(m):
        if (np.isnan(u[i])) or (np.isnan(v[i])):
            continue
            
        udotv += u[i] * v[i]
        u_norm += u[i] * u[i]
        v_norm += v[i] * v[i]

    u_norm = np.sqrt(u_norm)
    v_norm = np.sqrt(v_norm)
    
    if (u_norm == 0) or (v_norm == 0):
        ratio = 0 ## o podria devolver nan
    else:
        ratio = udotv / (u_norm * v_norm)
    return ratio

In [ ]:
## vamos a calcular la distancia usando pdist y squareform
## la primera vez que lo corramos va a ser un poco más lento
similitudes = squareform(pdist(user_item,metric=fast_cosine))

In [ ]:
## para comparar con la diapositiva: 
similitudes = np.round(similitudes,3)
similitudes

In [ ]:
similitudes

In [ ]:
## Si queremos predecir el score que Nico le da a Study:
nico = user_item[:,0]
study = similitudes[:,4]

np.nansum(nico*study)/np.sum(study[~np.isnan(nico)])

## otra opcion seria hacer el producto de la matriz user_item con la matriz de similitudes 
#luego al momento de sacar el rating predicho, calcular k y dividir

Veamos un ejemplo con datos reales: el dataset movielens

In [ ]:
mlens = pd.read_csv("u.data",sep="\t",header=None)
mlens.columns = ["user_id","item_id","rating","timestamp"]

In [ ]:
## primero vamos a mapear los ids de usuario y de pelicula a un nuevo id que sea un indice
user2ix = {user:ix for ix,user in enumerate(mlens.user_id.unique())}
ix2user = {user2ix[k]:k for k in user2ix.keys()}

item2ix = {item:ix for ix,item in enumerate(mlens.item_id.unique())}
ix2item = {item2ix[k]:k for k in item2ix.keys()}

mlens_con_ix = mlens.copy()
mlens_con_ix["user_id"] = mlens.user_id.apply(lambda x: user2ix[x])
mlens_con_ix["item_id"] = mlens.item_id.apply(lambda x: item2ix[x])

In [ ]:
## hacemos un pivot sobre el dataframe para tener la matriz usuario-item
## ojo, esta no es la forma mas eficiente: estamos usando un dataframe
## pero pdist y cdist no toman una sparse matrix como input
## de todas maneras, para este problema no requerimos una sparse matrix necesariamente
mlens_pivotada = mlens.pivot("item_id","user_id")["rating"]

In [ ]:
## calculamos la similitud
similitudes = squareform(pdist(mlens_pivotada,metric=fast_cosine))

In [ ]:
## vamos a usar una funcion que devuelva el rating original si este existe
## caso contrario, devuelve el predicho
def predecir(user,item,similitudes,original,user2ix,item2ix):
    ix_user = user2ix[user]
    ix_item = item2ix[item]
    score = original.iloc[ix_item,ix_user]
    if np.isnan(score):
        item_similitudes = similitudes[:,ix_item]
        user_scores = original.iloc[:,ix_user].values
        k = np.nansum(item_similitudes[~np.isnan(user_scores)]) ## este es el denominador de la funcion
        
        if k == 0: 
            ## si nadie le puso puntaje al item o las similitudes son todas 0 devuelvo un score arbitrario
            return 2.5
        
        score = np.nansum(user_scores*item_similitudes)
        score = score/k
    return score

In [ ]:
user = 554
item = 651
predecir(user,item,similitudes,mlens_pivotada,user2ix,item2ix)